In [1]:
import scanpy as sc

/diskmnt/Projects/Users/estorrs/miniconda3/envs/sc_scanpy_seurat/lib/python3.6/site-packages/anndata/core/anndata.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [3]:
adata = sc.read_10x_h5(
    '/diskmnt/Datasets/Emerson_Cervical/processed/SC_RNAseq/CESC_processed/TWDD-CE468-XB1_1_HPV/outs/raw_feature_bc_matrix.h5'
)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [4]:
adata

View of AnnData object with n_obs × n_vars = 6794880 × 33751 
    var: 'gene_ids', 'feature_types', 'genome'

In [5]:
adata.var

,gene_ids,feature_types,genome
host__MIR1302-2HG,host__ENSG00000243485,Gene Expression,host
host__FAM138A,host__ENSG00000237613,Gene Expression,host
host__OR4F5,host__ENSG00000186092,Gene Expression,host
host__AL627309.1,host__ENSG00000238009,Gene Expression,host
host__AL627309.3,host__ENSG00000239945,Gene Expression,host
...,...,...,...
virus_HPV75,virus_HPV75,Gene Expression,virus
virus_HPV78,virus_HPV78,Gene Expression,virus
virus_HPV-mCG2,virus_HPV-mCG2,Gene Expression,virus
virus_HPV-mCG3,virus_HPV-mCG3,Gene Expression,virus


In [7]:
adata = sc.read_10x_mtx(
    '/diskmnt/Projects/Users/estorrs/single_cell_data/test_matrix'
)

In [8]:
adata.var

,gene_ids,feature_types
MIR1302-2HG,ENSG00000243485,Gene Expression
FAM138A,ENSG00000237613,Gene Expression
OR4F5,ENSG00000186092,Gene Expression
AL627309.1,ENSG00000238009,Gene Expression
AL627309.3,ENSG00000239945,Gene Expression
...,...,...
HPV75,HPV75,Gene Expression
HPV78,HPV78,Gene Expression
HPV-mCG2,HPV-mCG2,Gene Expression
HPV-mCG3,HPV-mCG3,Gene Expression
